# Class three - Dealing with iteration

So far we've covered a lot of ground!  Talking about types and math and imports, My Word!  But the greatest strength of computers has not yet been discussed.  This foundational power house is central to why computers are the transformative power houses they are.  And all of it comes down to one simple idea: iteration.

To get a true sense of the real power of iteration, let's look at a historical example:

This comes to us from world war 2 and the work of Alan Turing, one of the first computer scientists.

He used iteration to crack the German's enigma machine.  The specifics of how we did it are out of scope for the course.  But I can say iteration is at the core of the solution.  

The problem being solved is well formulated in the movie the imitation game - here is an approximate description of that problem:

The German's send codes using engima over open airways.  Anyone can intercept them, but the messages will be encoded.  You need an engima machine in order to decode the message.  Now the Polish Cryptographers have figured out how to decode the engima machine, if they know the configuration of the machine.  Unfortunately, the configuration space for enigma was impossibly large:

In [4]:
60 * 17576 * 676 * 150738274937250

107458687327250619360000

possible configurations to be exact.  It is impossible to know which configuration to check, so you have to check all of them.  Before Turing built his machine, you'd need to do this _by hand_.

Here is some "semi" real code that does what turing's machine did.

In [ ]:
def combine(config, message):
    # actual config message is out of scope
    return message

def check_config(config, message):
    result = combine(config, message)
    if "Hitler" in result:
        return True
    else:
        return False
    
def find_config():
    for config in configurations:
        if check_config(config, message):
            return config
    return "config not found"

Granted, the combine function which I haven't defined is complex and enumerating all the possible configurations is also complex.  But other than that, the algorithm is really pretty simple, _because of iteration_.  That's what makes computers so powerful - they can iterate through an extremely large number of possible things, very, very fast.

This of course isn't the only example of iteration changing the way of humanity.  The next example comes to us from NASA in the 1960s.  

The problem definition here was also well formed in another major film - hidden figures:

The rocket they were trying to send into space needed dynamic calculations of certain forces.  If the calculations were wrong, on reentry, the space capsule would not come down correctly, killing the pilot inside.  So you needed a method to dynamically figure out the forces on the capsule, in order to dynamically readjust the forces on the capsule.

Enter Euler's method:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

# Concentration over time
N = lambda t: N0 * np.exp(-k * t)
# dN/dt
def dx_dt(x):
    return -k * x

k = .5
h = 0.001
N0 = 100.

t = np.arange(0, 10, h)
y = np.zeros(len(t))

y[0] = N0
for i in range(1, len(t)):
    # Euler's method
    y[i] = y[i-1] + dx_dt(y[i-1]) * h

max_error = abs(y-N(t)).max()
print('Max difference between the exact solution and Euler's approximation with step size h=0.001:')

print('{0:.15}'.format(max_error))

There is a lot of heavy math in this function, but don't focus on it.  Just recognize that there is a loop here:

```
for i in range(1, len(t)):
    # Euler's method
    y[i] = y[i-1] + dx_dt(y[i-1]) * h
```

Without iteration, we never would have been able to make these calculations fast enough to ensure good enough force calculations.

Hopefully this well motivates iteration for you and it's power - defeating the Nazi's in world war 2 and safely returning home from space travel.  That's just some of the power of iteration.

Enough talking about it!  Here's how to do it.

# The While Loop

The while loop is the simplest form of iteration and is easiest to reason about, so we'll start with that.  The general form of a while loop is:

1. intialize value
2. write while loop with exit loop condition
3. block of code to execute until loop completes

In [34]:
print("by hand")
x = 0
print(x)
x = x + 1
print(x)
x = x + 1
print(x)
print("using iteration")
x = 0
while x < 10:
    print(x)
    x = x + 1

by hand
0
1
2
using iteration
0
1
2
3
4
5
6
7
8
9


The above shows the simplicity of doing the computation.  Something important to note: we have to update the value of x in our loop.  If we didn't than `x < 10` would never become `False` and our loop would continue forever.  That's because while loops are kind of like `if` statements - they execute until they evaluate to `False`.

Let's rewrite our while loop with some short hand in the python language.

In [6]:
x = 0
while x < 10:
    print(x)
    x += 1 # equivalent to x = x + 1

0
1
2
3
4
5
6
7
8
9


Notice this code is more or less the same, the big difference is the shift from `x = x + 1` to `x += 1`.  It's kind of a weird thing in general and you have to get used to it, I certainly did.  What's actually happening is you are saying:

x + 1 and then store the result of that in a new variable which overwrites the old value of x.
So you actually read the code x = x + 1 from right hand side to left hand side, instead of left hand side to right hand side.  Kinda weird right?

The short hand and the original code: `x = x + 1` and `x += 1` mean the same thing.  There is no deep logical stuff going on here.  This is just a bit of syntax change for readability.

## Introduction to Lists

So we've talked about iteration, the next thing to cover is lists, so we can have something to iterate over!  A list is just a collection of objects stored together.  We can think of this kind of like a list of nouns.

Let's see an example of a human list:

* dough
* breed
* cereal
* rice
* pasta

This is a list of grains, something many of us should already be familiar with.  Here's another example:

* Eric
* Japan
* telephone
* 5
* False
* running

This is a list of words, with no inherent meaning.  So we can say, in general, a list is just a collection of things.  Whether they are associated or not doesn't matter, but we syntactically can group anything together, using english.  Now let's look at a Python list.

In [36]:
listing = [1, 2, 3, 4, 5, 6]
print(listing)
listing_two = list()
print(listing_two)

[1, 2, 3, 4, 5, 6]
[]


Here we see that we actually group Python nouns, like integers together!  The syntax for creating a list is as follows:

`variable_name = []`

OR

`variable_name = list()`

Both of the above create empty lists.  We can also start with lists that have elements in them!  

Like the above example, or this one:

In [3]:
second_list = ["hello", "there", "friends"]
second_list

['hello', 'there', 'friends']

A list can store collections of any Python basic type we might like!  Even ones we dream up, but more on that later.  

Let's see another example!

In [5]:
third_list = ["hello", 5, 7.2, True]
third_list

['hello', 5, 7.2, True]

We can even store multiple types in one list!  That's _crazy_!!!

Now let's look at adding an element to a list and taking one away.

In [7]:
listing = [1, 2, 3, 4, 5]
listing.append(6)
print("After calling the append method the list looks like this {}".format(listing))
listing.remove(2)
print("After calling the remove method the list looks like this {}".format(listing))

After calling the append method the list looks like this [1, 2, 3, 4, 5, 6]
After calling the remove method the list looks like this [1, 3, 4, 5, 6]


In the above example we added an element to the list, by appending it to the end of the list and we removed an element from the list, by calling the remove method.  Notice that append adds to the end of the list, while remove removes semantically from wherever the element in question is, in the list.

Let's look at the full list of possible operations on lists

In [10]:
[elem for elem in dir(list()) if "__" not in elem]

['append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [48]:
5 < 7 # these are comparable!
# [] < {}
listing = [5, "five"]
print(type(listing[0]))
print(type(listing[1]))
listing.insert(0, "happy")
del listing[0]
listing

<class 'int'>
<class 'str'>


[5, 'five']

One of the most powerful things about lists is, if they are all the same type and comparable to one another, then you can call `list.sort()`, which will sort the elements in your list from lowest to highest for you.  Sorting is out of scope for this course, but I'll say, the sorting algorithm Python uses is very, very fast.

Some other important methods are `count`, which counts the number of occurrences of an element in the list:

In [40]:
listing = [1, 2, 1, 2, 3, 3, 3, 3]
listing.count(1)

2

The `index` method which tells you the first index of an element.

In [42]:
listing = [1, 2, 3, 4, 5, 1]
listing.index(1)

0

So at this point you may be asking yourself, what's an index?  An index is a number that allows you to call a specific element, by it's place in the list.  Often times the way programmers talk about this is by saying "indexing into the list at a specific element".  This way of describing it makes sense of you look at syntactically you reference an element by it's index in the list:

In [15]:
listing = [1, 2, 3, 4, 5, 6, 7, 8]
print("The zeroth index of the list is {}".format(listing[0]))
print("The 3rd index of the list is {}".format(listing[3]))

The zeroth index of the list is 1
The 3rd index of the list is 4


## An introduction to for loops

Notice that the first index or first place in the list is 0, not 1.  This is a point of confusion for lots of folks, but number theorists have insisted for a long time that counting should start at 0, not 1.  There are some reasons this can be nice, for calculating distances in discrete space.  

The next thing we'll look at is making use of our indexing to iterate over the list.  This will be just like what we did with while loops, except with a difference syntax.  First we'll see the example with a while loop and then we'll see it with a for loop.

In [19]:
listing = [1, 2, 3, 4, 5, 6, 7, 8]

print("Iterating with a while loop")
iterator = 0
while iterator < len(listing):
    print(listing[iterator])
    iterator += 1
print()
print("Interating with a for loop")
for elem in listing:
    print(elem)

Iterating with a while loop
1
2
3
4
5
6
7
8

Interating with a for loop
1
2
3
4
5
6
7
8


As you can see the two ways of looping are equivalent.  However, the for loop is the preferred way of doing this.  At first, at may seem less explicit, but you'll find that it leads to fewer errors in the program.  This is because you don't need to explicitly set the boolean condition for your loop to terminate.  Infinite loops - loops that never end are a big problem for while loops, and sometimes be very, very hard to debug, depending on how complex your terminating condition is.

For loops solve this nicely, because you don't tell the loop when to terminate, the for loop figures it out, because it just goes over every index in the list.

If you want to do something sophisticated with your list, which some folks do, you can explicitly still set up the counter like we did before, with the for-loop, so you don't lose any of the expressive power of the while loop, while using the for loop.

In [22]:
iterator = 0
listing = [1, 2, 3, 4, 5, 6, 7, 8]

for elem in listing:
    print("The element of the list at index {} is {}".format(iterator, elem))
    iterator += 1

The element of the list at index 0 is 1
The element of the list at index 1 is 2
The element of the list at index 2 is 3
The element of the list at index 3 is 4
The element of the list at index 4 is 5
The element of the list at index 5 is 6
The element of the list at index 6 is 7
The element of the list at index 7 is 8


This is a powerful and flexible syntax, but having to initialize the interator is also kind of annoying, so Python comes with a nice function doing this for you automatically as well:

In [50]:
listing = [1, 2, 3, 4, 5, 6, 7, 8]
for iterator, elem in enumerate(listing):
    print("The element of the list at index {} is {}".format(iterator, elem))

The element of the list at index 0 is 1
The element of the list at index 1 is 2
The element of the list at index 2 is 3
The element of the list at index 3 is 4
The element of the list at index 4 is 5
The element of the list at index 5 is 6
The element of the list at index 6 is 7
The element of the list at index 7 is 8


No more pesky updating of iterators!  Now everything is clear, high level and clean.  

So with all this talk of indexing, you may wonder, do you always have to index your lists by the counting numbers, also known as the natural numbers?

## An introduction to dictionaries

The answer is no!  It's time to meet our next kind of collection - the dictionary!

In [24]:
dicter = {1:2, 2:3, 4:7}
dicter

{1: 2, 2: 3, 4: 7}

In this dictionary, the index is the first number and the values are the second number.  In this collection we defined the index to be 1, 2, and 4 and the stored values to be 2, 3, and 7.  Certainly not indexed by the counting numbers, incremented one at a time!

We can do a lot more than just have a weird indexing strategy with dictionaries.  We can _even_ create mappings or lookups from words to other words.


In [26]:
dicter_two = {"Eric": "Teacher", "Bob": "Person", "Dog":"Cat"}
sentence = "Hi my name is Eric I'm friends with Bob and he has a Dog named Dog"
new_sentence = []
for word in sentence.split():
    if word in dicter_two:
        new_sentence.append(dicter_two[word])
    else:
        new_sentence.append(word)
" ".join(new_sentence)

"Hi my name is Teacher I'm friends with Person and he has a Cat named Cat"

There is a lot going on here!  The first thing to see is that we can create a list, by calling `split()` on a string!  This method is pretty amazing, it splits strings on white space characters like `' '`.  And turns them into lists of words.  Then I'm using a for loop for each word, if the word is in the index, usually called keys, of the dictionary then I do the mapping.  Otherwise I append the word to the list, as is.  Finally a join all the words back together.

The join method on a string is the opposite operation of split.  Instead of splitting apart a string into a bunch of strings in a list.  It takes a list of strings and stitches them all together with whatever string you are calling join on, in this case, a white space.

## An introduction to sets
The final thing we are going to discuss today is sets.

A set is the least interesting, but still useful, collection we will be looking at today.  You can't index into it, you can't even add more than one element with the same value to it!

But this turns out to be a powerful tool none the less.  Here we will see some of the power of sets:

In [29]:
import random
listing = [random.randint(1,10) for _ in range(10000)]
print("This list has {} elements of which {} are unique".format(len(listing), len(set(listing))))

This list has 10000 elements of which 10 are unique


We can also iterate over sets like follows:


In [30]:
set_list = set(listing)
for elem in set_list:
    print(elem)

1
2
3
4
5
6
7
8
9
10
